In [78]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

Single trade each day version, 1.25MW battery (nominal charge 20% or 0.25 MW)

In [139]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_B =  current_Q_B[(current_Q_B.index < max_price_index)]
    
        if len(remaining_prices_B) > 0:
            max_price_index = max_price_index
            min_price_index = remaining_prices_B['Price'].idxmin()
            if (((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2))>0:
         
                    profit = ((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2)
            
                    prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
            
            elif (((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2))<0:
            
                     profit = ((current_df.loc[max_price_index, 'Price'])) - ((current_df.loc[min_price_index, 'Price'])/eff_2)
            
                     prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
            
            else:
                continue  
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)

In [143]:
print(sum(r_dam_50_50.iloc[:,4:5].values))
r_dam_50_50

[16790.66714286]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,5,12.26,17,31.66,12.817796
1,28,12.89,42,36.00,15.646939
2,50,16.23,65,37.16,13.166776
3,85,15.00,90,37.00,14.293878
4,99,17.40,104,35.19,10.396898
...,...,...,...,...,...
356,8640,88.64,8659,180.29,53.783020
357,8667,86.24,8673,219.04,87.232000
358,8692,87.65,8696,252.37,112.457224
359,8715,77.62,8721,128.52,23.611918


In [144]:
print(sum(r_dam_10_90.iloc[:,4:5].values))
r_dam_10_90

[16654.89677551]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,25,12.64,42,36.00,15.902041
1,50,16.23,66,37.30,13.278776
2,85,15.00,91,33.17,11.229878
3,98,17.12,104,35.19,10.682612
4,123,-1.00,132,2.12,2.716408
...,...,...,...,...,...
355,8643,86.00,8659,180.29,56.476898
356,8667,86.24,8672,246.11,108.888000
357,8691,85.31,8696,252.37,114.844980
358,8715,77.62,8731,117.03,14.419918


In [145]:
print(sum(r_dam_30_70.iloc[:,4:5].values))
r_dam_30_70

[17005.27330612]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,4,11.58,18,28.66,11.111673
1,25,12.64,42,36.00,15.902041
2,50,16.23,65,37.16,13.166776
3,85,15.00,90,37.00,14.293878
4,98,17.12,104,35.19,10.682612
...,...,...,...,...,...
356,8643,86.00,8659,180.29,56.476898
357,8667,86.24,8672,246.11,108.888000
358,8691,85.31,8696,252.37,114.844980
359,8715,77.62,8731,117.03,14.419918


In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)


In [2]:
column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 23)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 23)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 23)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 23)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 23)]
Y_r=dat1[column_names].dropna().stack().reset_index()

# plt.plot(Y_r.iloc[:, 2:])
# Y_r.iloc[:, 2:].shape
# Q_30

In [4]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        max_profit = 0
        best_buy_timestamp = i
        best_sell_timestamp = i

        for j in range(i, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue

            for k in range(j, min(i+24, len(quantile_70_predictions))):
                if k - j > 48:
                    break
                if quantile_70_predictions[j] < quantile_30_predictions[k]:
                    purchase_price = real_prices[j]
                    sell_price = real_prices[k]
                    units = min(max_units, abs(funds // purchase_price))
                    if purchase_price < 0:
                        units = max_units
                    profit = (units * (sell_price - purchase_price)) * 0.8

                    if profit > max_profit:
                        max_profit = profit
                        best_buy_timestamp = j
                        best_sell_timestamp = k

        if max_profit > 0:
            purchase_price = real_prices[best_buy_timestamp]
            sell_price = real_prices[best_sell_timestamp]
            units = min(max_units, abs(funds // purchase_price))
            if purchase_price < 0:
                units = max_units
            profit = (units * (sell_price - purchase_price)) * 0.8

            profits += profit
            funds = funds 

            trade_log.append((best_buy_timestamp, best_sell_timestamp, purchase_price, sell_price, units, profit))

            num_trades += 1
            latest_buy_timestamp = best_buy_timestamp

    return profits, trade_log, num_trades, units, funds



quantile_30_predictions = np.array(pd.DataFrame(Q_30.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_70.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

[192029.12]
350
[3500]


/tmp/ipykernel_34763/2301800345.py:25: RuntimeWarning: divide by zero encountered in floor_divide
  units = min(max_units, abs(funds // purchase_price))
/tmp/ipykernel_34763/2301800345.py:38: RuntimeWarning: divide by zero encountered in floor_divide
  units = min(max_units, abs(funds // purchase_price))


In [5]:
pd.DataFrame(trade_log).head(n=39)

,0,1,2,3,4,5
0,4,16,[11.58],[31.66],10,[160.64]
1,26,40,[10.26],[39.82],10,[236.48000000000002]
2,48,64,[16.23],[37.3],10,[168.55999999999997]
3,81,87,[15.0],[37.0],10,[176.0]
4,96,99,[18.55],[35.22],10,[133.35999999999999]
5,120,132,[-1.0],[21.49],10,[179.92]
6,151,158,[-27.93],[13.56],10,[331.9200000000001]
7,168,178,[-1.1],[46.03],10,[377.04]
8,197,214,[12.95],[37.0],10,[192.4]
9,220,224,[23.89],[41.69],10,[142.39999999999998]


In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        max_profit = 0
        best_buy_timestamp = i
        best_sell_timestamp = i

        for j in range(i, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue

            for k in range(j, min(i+24, len(quantile_70_predictions))):
                if quantile_70_predictions[j] < quantile_30_predictions[k]:
                    purchase_price = real_prices[j]
                    sell_price = real_prices[k]
                    units = min(max_units, abs(funds // purchase_price))
                    profit = units * (sell_price - purchase_price) * 0.8

                    if profit > max_profit:
                        max_profit = profit
                        best_buy_timestamp = j
                        best_sell_timestamp = k

        if max_profit > 0:
            purchase_price = real_prices[best_buy_timestamp]
            sell_price = real_prices[best_sell_timestamp]
            units = min(max_units, abs(funds // purchase_price))
            profit = units * (sell_price - purchase_price) * 0.8

            profits += profit
            funds = funds

            trade_log.append((best_buy_timestamp, best_sell_timestamp, purchase_price, sell_price, units, profit))

            num_trades += 1
            latest_buy_timestamp = best_buy_timestamp

    return profits, trade_log, num_trades, units, funds



def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        max_profit = 0
        best_buy_timestamp = i
        best_sell_timestamp = i

        for j in range(i, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue

            if quantile_70_predictions[best_buy_timestamp] < quantile_30_predictions[j]:
                purchase_price = real_prices[best_buy_timestamp]
                sell_price = real_prices[j]
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price) * 0.8

                if profit > max_profit:
                    max_profit = profit
                    best_sell_timestamp = j

        if max_profit > 0:
            purchase_price = real_prices[best_buy_timestamp]
            sell_price = real_prices[best_sell_timestamp]
            units = min(max_units, abs(funds // purchase_price))
            profit = units * (sell_price - purchase_price) * 0.8

            profits += profit
            funds = funds

            trade_log.append((best_buy_timestamp, best_sell_timestamp, purchase_price, sell_price, units, profit))

            num_trades += 1
            latest_buy_timestamp = best_buy_timestamp
            

    return profits, trade_log, num_trades, units, funds



def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        for j in range(i+1, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue
            if quantile_70_predictions[i] < quantile_30_predictions[j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[j]
                
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price) * 0.8
                
                profits += profit
                funds += profit
                
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                latest_buy_timestamp = i
                
                break

    return profits, trade_log, num_trades, units, funds

In [45]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        for j in range(i+1, min(i+24, len(quantile_70_predictions))):
            if quantile_70_predictions[i] < quantile_30_predictions[j]:
                
                purchase_price = real_prices[i]
                sell_price = real_prices[j]
                
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price) * 0.8
                
                profits += profit
                funds = funds + profit
                
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = j
                
                break

    return profits, trade_log, num_trades, units, funds




quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))
# print(units)
# print(funds)
# sum(pd.DataFrame(trade_log).iloc[:, 5:].values)

[1480.224]
85
[array([849.])]


/tmp/ipykernel_228461/2279083772.py:17: RuntimeWarning: divide by zero encountered in floor_divide
  units = min(max_units, abs(funds // purchase_price))


In [21]:
pd.DataFrame(trade_log).head(n=20)

,0,1,2,3,4,5
0,0,5,[-18.28],[-20.43],10,[-17.19999999999999]
1,24,25,[36.62],[30.89],10,[-45.839999999999975]
2,48,49,[-5.27],[-2.37],10,[23.199999999999996]
3,72,73,[9.04],[16.0],10,[55.68000000000001]
4,96,97,[42.3],[51.81],10,[76.08000000000004]
5,120,121,[-4.0],[22.41],10,[211.28000000000003]
6,144,145,[39.7],[50.8],10,[88.79999999999995]
7,168,169,[59.0],[53.96],10,[-40.31999999999999]
8,192,199,[84.0],[60.19],10,[-190.48000000000002]
9,216,217,[56.0],[57.03],10,[8.240000000000009]


array([array([726.])], dtype=object)

,4
0,10
1,10
2,10
3,10
4,10
...,...
71,10
72,10
73,10
74,[4.0]


working attempt where we can restrict the appraoch to purchase when the 70th quantile prediction at time point i is less than the 30th quantile prediction at a later time point i+j.

No optimisation used here, this is just representative of a working example. Optimisation approach included further down.

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price)*0.8
                
                profits += profit
                funds = funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds


quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_70.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
# print(units)
# print(funds)
# sum(pd.DataFrame(trade_log).iloc[:, 5:].values)

In [ ]:
pd.DataFrame(trade_log)

In [ ]:
sum(pd.DataFrame(trade_log).iloc[:, 5:].values)

In [ ]:
pd.DataFrame(trade_log)

Of interest, increasing the funds also changes the multiple of profits, likely as funds is not constrained by unit
size or quantity like profits are, ignoring our effect on the price.

what I'll do is collect predictions for the DAM over the same time period with the same method to see if it
makes a sunstantial difference on either trade frequence or profits levels compared to the BM

potentiall because the market is so volatile the models are learning to take a wider position on uncertainty 
forcing them to trade far less. This potentially may not be the case in the DAM allowing us to trade with greater frequency but will require a constraint that all trading periods are within each 24 hour time frame




Pasted Below is a comparison of trading off of all different quantile pairs. No constraint on maximum units, first row represents total profits, second row total trades, third row number of units purchased


In [ ]:
#0.1,0.3
[2800.17]
8
[12.]
#0.1,0.5
[700.68]
5
[3.]
#0.1,0.7
[3645.99]
4
[15.]
#0.1,0.9
[1117.]
2
[9.]

#0.3,0.5
[1588.93]
9
[7.]
#0.3,0.7
[1099.07]
14
[5.]
#0.3,0.9
[6202.04]
6
[25.]

#0.5,0.7
[2516.54]
16
[11.]
#0.5,0.9
[33864.86]
11
[133.]

#0.7,0.9
[4609.31]
17
[20.]

Pasted Below: maximum units 1, first row total profits, second row total trades


In [ ]:
#0.1,0.3
[405.58]
8
#0.1,0.5
[320.14]
5
#0.1,0.7
[304.89]
4
#0.1,0.9
[377.55]
2

#0.3,0.5
[484.33]
9
#0.3,0.7
[356.7]
14
#0.3,0.9
[429.75]
6

#0.5,0.7
[368.96]
16
#0.5,0.9
[854.84]
11

#0.7,0.9
[532.49]
17

Taken from google on the effects of the optimsation approach.

Using optimisation even though there will be no change in the output with the current weights of [1,1],however varying these constraints 'x0' which represents the initial value for the optimization algorithm's decision variables. In this case, the decision variables are the weights [1, 1]. The optimization algorithm will adjust these weights to maximize the profits returned by the calculate_profits() function.

The weights determine the criteria for when to buy and sell. If the weights are set to a certain value, the algorithm will be more or less likely to trade based on those criteria. For example, if w_0 is set to a very high value, the algorithm may be more likely to trade because it is taking into account the predictions of the upper quantile more heavily. This could result in a higher trading frequency.

This may be well suited to a hyper paramater search for optimal weighting for the trading portion.

In [ ]:
import numpy as np
from scipy.optimize import minimize

def calculate_profits(weights, quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 1

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] * weights[0] < quantile_30_predictions[i+j] * weights[1]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                
                profits += profit
                funds = (sell_price/purchase_price) * funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds

#lower quantile
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
#upper quantile
quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))


def objective(weights, quantile_70_predictions, quantile_30_predictions, real_prices):
    profits, trade_log, num_trades, units, funds = calculate_profits(weights, quantile_70_predictions, quantile_30_predictions, real_prices)
    return -profits

x0 = [3, 1]

res = minimize(objective, x0, args=(quantile_70_predictions, quantile_30_predictions, real_prices), method='SLSQP')

profits, trade_log, num_trades, units, funds = calculate_profits(x0, quantile_70_predictions, quantile_30_predictions, real_prices)

# print(res.x)
print(-res.fun)
print(num_trades)
print(units)
# print(profits)
trade_log


With a added threshold paramater added so to only trade when expected profit or loss is within a certain range

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices, threshold):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 1

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            purchase_price = real_prices[i]
            sell_price = real_prices[i+j]
            expected_profit = (quantile_30_predictions[i+j] - quantile_70_predictions[i]) / quantile_70_predictions[i]

            # Only execute trade if expected profit or loss is within the threshold
            if quantile_70_predictions[i] < quantile_30_predictions[i+j] and abs(expected_profit) <= threshold:
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                
                profits += profit
                funds = (sell_price/purchase_price) * funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds


quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices, 0.1)
print(total_profits)
print(num_trades)
print(units)
print(funds)

trade_log

In [ ]:
#pymo

Plot below should highlight its a very rare occurence for the 0.9 quantile at time point i to be below the 0.1 quantile at time point i+j.

Point stands for quantile 0.7 and 0.3

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl

plt.style.use('plot_style.txt')
#=========================================================== # Directory and filename; style file open #=========================================================== # Change to the directory which contains the current script 
dirFile = os.path.dirname(os.path.join('YourDirHere',
                          'NicePlotProductivity.py'))
# Load style file plt.style.use(os.path.join(dirFile, 'PaperDoubleFig.mplstyle'))
# Make some style choices for plotting 
colourWheel =['#329932',
            '#ff6961',
            'b',
            '#6a3d9a',
            '#fb9a99',
            '#e31a1c',
            '#fdbf6f',
            '#ff7f00',
            '#cab2d6',
            '#6a3d9a',
            '#ffff99',
            '#b15928',
            '#67001f',
            '#b2182b',
            '#d6604d',
            '#f4a582',
            '#fddbc7',
            '#f7f7f7',
            '#d1e5f0',
            '#92c5de',
            '#4393c3',
            '#2166ac',
            '#053061']
dashesStyles = [[3,1],
            [1000,1],
            [2,1,10,1],
            [4, 1, 1, 1, 1, 1]]





p1=Q_10.iloc[:, 2:]
p1=p1.reset_index(drop=True)
p3=Q_30.iloc[:, 2:]
p3=p3.reset_index(drop=True)
p5=Q_50.iloc[:, 2:]
p5=p5.reset_index(drop=True)
p7=Q_70.iloc[:, 2:]
p7=p7.reset_index(drop=True)
p9=Q_90.iloc[:, 2:]
p9=p9.reset_index(drop=True)
yy=Y_r.iloc[:, 2:]
yy=yy.reset_index(drop=True)

p1=p1.values.ravel()
p3=p3.values.ravel()
p5=p5.values.ravel()
p7=p7.values.ravel()
p9=p9.values.ravel()
                       
figure(figsize = (30, 20), dpi = 90)
plt.plot(yy, color='black', label='BM prices', alpha=0.3)
plt.plot(p1, color='blue', label='predictions 0.1 quantile', alpha=0.3)
# plt.plot(p3, color='yellow', label='SH DNN predictions 0.3 quantile')
# plt.plot(p5, color='red', label='SH DNN predictions 0.5 quantile')
# plt.plot(p7, color='yellow', label='SH DNN predictions 0.7 quantile')
plt.plot(p9, color='blue', label='predictions 0.9 quantile', alpha=0.3)
plt.fill_between(np.arange(0,len(yy),1), p1, p9, alpha=0.2)


plt.xlabel('0.1, 0.9 quantiles Forecast period from 01/06/20-01/06/21',fontsize=18)
plt.ylabel('€/MWh', fontsize=18)
plt.title("Forecast of BM prices", fontsize=18)
plt.xlim([-10, 3000])
plt.ylim([-50,200])
plt.legend()
# plt.savefig('SH_DNN.png', transparent=False, format="jpg")


In [ ]:
figure(figsize = (30, 20), dpi = 90)
plt.plot(yy, color='black', label='BM prices', alpha=0.3)
plt.plot(p3, color='blue', label='predictions 0.3 quantile', alpha=0.3)
# plt.plot(p3, color='yellow', label='SH DNN predictions 0.3 quantile')
# plt.plot(p5, color='red', label='SH DNN predictions 0.5 quantile')
# plt.plot(p7, color='yellow', label='SH DNN predictions 0.7 quantile')
plt.plot(p7, color='blue', label='predictions 0.7 quantile', alpha=0.3)
plt.fill_between(np.arange(0,len(yy),1), p3, p7, alpha=0.2)


plt.xlabel('0.3, 0.7 quantiles Forecast period from 01/06/20-01/06/21',fontsize=18)
plt.ylabel('€/MWh', fontsize=18)
plt.title("Forecast of BM prices", fontsize=18)
plt.xlim([-10, 3000])
plt.ylim([-50,200])
plt.legend()
# plt.savefig('SH_DNN.png', transparent=False, format="jpg")


Everthing below contains previous attempt at the current working code or is junk working copies adapted from articles on trading.

In [ ]:
# def objective_function(w, q30, q70, prices):
#   profits = []

#   last_trade_timestamp = -1

#   trade_threshold = 0.5

#   for i, price in enumerate(prices[:-1]):
#     for j in range(1, len(q30) - i):
#       if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
#         if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
#           purchase_price = prices[i]
#           sell_price = prices[i + j]

#           if sell_price > (1 + trade_threshold) * purchase_price:
#             continue  

#           profit = sell_price - purchase_price

#           profits.append(profit)
#           global funds, trades  
#           funds = funds + (sell_price - purchase_price)
#           trades += 1 
        
#           last_trade_timestamp = i + j

#   total_profit = sum(profits)


#   return -total_profit
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 1
    trade_threshold = 0.9


    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                if sell_price > (1 + trade_threshold) * purchase_price:
                    continue 
                
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                
                profits += profit
                funds = (sell_price/purchase_price) * funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds

quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(units)
print(funds)

trade_log

In [ ]:
# dat1 = df.loc[(df['lag_2y'] < -5) | (df['lag_2y'] > 5)]
# dat1 = df.loc[(df.iloc[:,:16] > 5)]
# dat1 = df.loc[(df.iloc[:,:16] > 1).all(axis=1)]
dat.shape

dat1=df

In [ ]:
def optimize_trades(quantile_70_predictions, quantile_30_predictions, real_prices, max_units):
    prob = pulp.LpProblem('Trade Optimization', pulp.LpMaximize)
    
    x = pulp.LpVariable.dicts('trade', range(len(quantile_70_predictions)), lowBound=0, upBound=1, cat='Binary')

    prob += pulp.lpSum([x[i] * (real_prices[i+j] - real_prices[i]) for i in range(len(quantile_70_predictions)) for j in range(1, len(quantile_70_predictions)-i) if quantile_70_predictions[i] < quantile_30_predictions[i+j]])
    prob += pulp.lpSum([x[i] for i in range(len(quantile_70_predictions))]) <= max_units

    status = prob.solve()
    print(pulp.LpStatus[status])
    for v in prob.variables():
        print(v.name, '=', v.varValue)
    print('Total profits =', pulp.value(prob.objective))

optimize_trades(quantile_70_predictions, quantile_30_predictions, real_prices, 10)



In [ ]:
# Declare the funds and trades variables as global variables
funds = 1000
trades = 0
trade_log = []  # Declare a list variable to store the trades

def objective_function(w, q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  max_units = 1

  global funds, trades

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  # Check that the sell time stamp is after the last trade timestamp
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]
          
#           if sell_price <= 0:
#             continue

          units = min(max_units, funds // purchase_price)
          profit = units * (sell_price - purchase_price)
            
#           if profit <= 0:
#             continue

          profits.append(profit)
          funds += profit
          trades += 1
          trade_log.append((purchase_price, sell_price, units, profit))


          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit




df_q30 = pd.DataFrame(Q_10.iloc[:, 2:])
df_q70 = pd.DataFrame(Q_90.iloc[:, 2:])
df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

# Test the objective function with the DataFrame objects
w0 = [1, 1]  # Initial values for the model parameters
res = minimize(objective_function, w0, args=(np.array(df_q30), np.array(df_q70), np.array(df_prices)))

# Print the optimized model parameters, the final value of the funds, and the number of trades
print(f'Optimized model parameters: {res.x}')
print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')
print(f'Number of trades: {trade_log}')

In [ ]:
trade_log

In [ ]:
# Declare the funds and trades variables as global variables
funds = 100
trades = 0
trade_log = []  # Declare a list variable to store the trades

def objective_function(q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  max_units = 1

  global funds, trades

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]
          
          units = min(max_units, funds // purchase_price)
          profit = units * (sell_price - purchase_price)
            
          profits.append(profit)
          funds += profit
          trades += 1

          trade_log.append((purchase_price, sell_price, units, profit))
          
          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit


def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices, max_units):
    funds = 100
    profits = 0
    trade_log = []

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                profits += profit
                funds = (sell_price/purchase_price) * funds
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                num_trades += 1
                last_sell_timestamp = i+j
                break

    return profits




# df_q30 = pd.DataFrame(Q_10.iloc[:, 2:])
# df_q70 = pd.DataFrame(Q_90.iloc[:, 2:])
# df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

df_q30 =    [100, 110, 120, 130, 140, 150, 160]
df_q70 =    [90,  100, 150, 120, 190, 140, 190]
df_prices = [100,  60, 120, 30, 140, 150, 60]

objective_function(np.array(df_q30), np.array(df_q70), np.array(df_prices))

print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')

# print(f'Trade log: {trade_log}')


In [ ]:
trade_log
#purchase_price, sell_price, units, profit

In [ ]:
trade_log

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 100
    profits = 0
    trade_log = []

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                profits += sell_price - purchase_price
                funds = (sell_price/purchase_price) * funds
                trade_log.append((i, j, purchase_price, sell_price))
                num_trades += 1
                last_sell_timestamp = i+j
                break

    return profits, trade_log, num_trades


quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(trade_log)
print(num_trades)


In [ ]:
trade_log

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 100
    profits = 0
    trade_log = []
    max_units = 1


    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                profits += profit
                funds = (sell_price/purchase_price) * funds
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                num_trades += 1
                last_sell_timestamp = i+j
                break

    return profits, trade_log, num_trades

quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
# print(trade_log)
print(num_trades)
trade_log

In [ ]:
trade_log

In [ ]:
funds = 1000
trades = 0

def objective_function(w, q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  trade_threshold = 0.5

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]

          if sell_price > (1 + trade_threshold) * purchase_price:
            continue  

          profit = sell_price - purchase_price

          profits.append(profit)
          global funds, trades  
          funds = funds + (sell_price - purchase_price)
          trades += 1 
        
          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit

df_q30 = pd.DataFrame(Q_30.iloc[:, 2:])
df_q70 = pd.DataFrame(Q_70.iloc[:, 2:])
df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

w0 = [1, 1]  
res = minimize(objective_function, w0, args=(np.array(df_q30), np.array(df_q70), np.array(df_prices)))

print(f'Optimized model parameters: {res.x}')
print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')

In [ ]:

funds = 1000
trades = 0

def objective_function(w, q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]

          profit = sell_price - purchase_price

          profits.append(profit)
          global funds, trades  
          funds = (sell_price / purchase_price) * funds
          trades += 1  
        
          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit



df_q30 = pd.DataFrame(Q_10.iloc[:, 2:])
df_q70 = pd.DataFrame(Q_90.iloc[:, 2:])
df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

w0 = [1, 1]  
res = minimize(objective_function, w0, args=(np.array(df_q30), np.array(df_q70), np.array(df_prices)))

print(f'Optimized model parameters: {res.x}')
print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')

In [ ]:
p1=Q_10.iloc[:, 2:].values.ravel()
yy= Y_r.iloc[:, 2:].values.ravel()
p9=Q_90.iloc[:, 2:].values.ravel()
                       
figure(figsize = (30, 20), dpi = 90)
plt.plot(yy, color='black', label='BM prices', alpha=0.6)
plt.plot(p1, color='blue', label='SH DNN predictions 0.1 quantile', alpha=0.2)
# plt.plot(p3, color='yellow', label='SH DNN predictions 0.3 quantile')
# plt.plot(p5, color='red', label='SH DNN predictions 0.5 quantile')
# plt.plot(p7, color='yellow', label='SH DNN predictions 0.7 quantile')
plt.plot(p9, color='blue', label='SH DNN predictions 0.9 quantile', alpha=0.2)
plt.fill_between(np.arange(0,len(yy),1), p1, p9, alpha=0.4)
